In [7]:
import pandas as pd
import numpy as np


In [8]:
from scipy.sparse import csr_matrix

In [ ]:
events_df = pd.read_csv('input/events.csv')
category_tree_df = pd.read_csv('input/category_tree.csv')
item_properties_1_df = pd.read_csv('input/item_properties_part1.csv')
item_properties_2_df = pd.read_csv('input/item_properties_part2.csv')

In [ ]:
len(category_tree_df["parentid"].unique())

In [ ]:
len(events_df["visitorid"].unique())/len(events_df["visitorid"])

In [ ]:
len(item_properties_1_df.loc[(item_properties_1_df.property == 'categoryid') ].sort_values('timestamp')["value"].unique())

In [ ]:
data=events_df[events_df["event"]=="transaction"]

In [ ]:
visitors=events_df["visitorid"].unique()

In [ ]:
len(visitors)

In [ ]:
items=events_df["itemid"].unique()

In [ ]:
data_2=data.groupby(['visitorid']).head(50)

In [ ]:
data_2['visitors'] = data_2['visitorid'].apply(lambda x : np.argwhere(visitors == x)[0][0])
data_2['items'] = data_2['itemid'].apply(lambda x : np.argwhere(items == x)[0][0])

In [ ]:
len(data_2["items"].unique())

In [ ]:
occurences = csr_matrix((visitors.shape[0], items.shape[0]), dtype='int8')

In [ ]:
def set_occurences(visitor, item):
    occurences[visitor, item] += 1

In [ ]:
data_2.apply(lambda row: set_occurences(row['visitors'], row['items']), axis=1)

In [ ]:
occurences

In [ ]:
cooc = occurences.transpose().dot(occurences)
cooc.setdiag(0)

In [ ]:
cooc

In [ ]:
# the cooc matrix stores how many times both items are bought together by the user.

In [ ]:
def xLogX(x):
    return x * np.log(x) if x != 0 else 0.0

In [ ]:
def entropy(x1, x2=0, x3=0, x4=0):
    return xLogX(x1 + x2 + x3 + x4) - xLogX(x1) - xLogX(x2) - xLogX(x3) - xLogX(x4)

In [ ]:
def LLR(k11, k12, k21, k22):
    rowEntropy = entropy(k11 + k12, k21 + k22)
    columnEntropy = entropy(k11 + k21, k12 + k22)
    matrixEntropy = entropy(k11, k12, k21, k22)
    if rowEntropy + columnEntropy < matrixEntropy:
        return 0.0
    return 2.0 * (rowEntropy + columnEntropy - matrixEntropy)

In [ ]:
def rootLLR(k11, k12, k21, k22):
    llr = LLR(k11, k12, k21, k22)
    sqrt = np.sqrt(llr)
    if k11 * 1.0 / (k11 + k12) < k21 * 1.0 / (k21 + k22):
        sqrt = -sqrt
    return sqrt

In [ ]:
row_sum = np.sum(cooc, axis=0).A.flatten()
column_sum = np.sum(cooc, axis=1).A.flatten()
total = np.sum(row_sum, axis=0)
pp_score = csr_matrix((cooc.shape[0], cooc.shape[1]), dtype='double')
cx = cooc.tocoo()
for i,j,v in zip(cx.row, cx.col, cx.data):
    if v != 0:
        k11 = v
        k12 = row_sum[i] - k11
        k21 = column_sum[j] - k11
        k22 = total - k11 - k12 - k21
        pp_score[i,j] = rootLLR(k11, k12, k21, k22)

In [1]:
import pickle


In [2]:
filename='pp_score'

In [ ]:
with open(filename, "wb") as f:
    pickle.dump(pp_score, f)

In [4]:
with open(filename, "rb") as f:
    pp_score = pickle.load(f)

In [37]:
pp_score.A

MemoryError: Unable to allocate array with shape (235061, 235061) and data type float64